In [ ]:
# === CELL 1: LOAD MFCC FEATURES ===
# Run 01_MFCC_Extraction.ipynb first
X = np.load('mfcc_features.npy') 
y = np.load('labels.npy')

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print(f"Dataset: {X.shape[0]} samples")

# === CELL 2: ULTRA-LIGHTWEIGHT CNN (85KB) ===
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(13,)),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(3, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=50, batch_size=16, validation_split=0.2, verbose=1)

print(f"Test Accuracy: {model.evaluate(X_test, y_test)[1]:.1%}")

# === CELL 3: TFLITE EXPORT ===
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

with open('model/alertvibe.tflite', 'wb') as f:
    f.write(tflite_model)
    
import os
print(f"Model size: {os.path.getsize('model/alertvibe.tflite')/1024:.0f}KB")
